In [9]:
import tensorflow as tf

out_dir = "logs/embeddings"
ckpt_path = tf.train.latest_checkpoint(out_dir)
if not ckpt_path:
    raise FileNotFoundError(f"No checkpoint found under {out_dir}. "
                            "Check the folder and filenames (e.g. embedding.ckpt-1.*).")
print("Using checkpoint:", ckpt_path)

vars_in_ckpt = tf.train.list_variables(ckpt_path)
print("Variables in checkpoint:")
for name, shape in vars_in_ckpt:
    print(" ", name, shape)

candidate = None
candidate_shape = None
largest_size = 0
for name, shape in vars_in_ckpt:
    if len(shape) == 2:
        size = shape[0] * shape[1]
        if size > largest_size:
            largest_size = size
            candidate = name
            candidate_shape = shape

if candidate is None:
    raise RuntimeError("No 2-D variable found in checkpoint. Check saved checkpoint contents above.")

print(f"Loading variable '{candidate}' with shape {candidate_shape} from checkpoint...")
embeddings = tf.train.load_variable(ckpt_path, candidate)  # returns numpy array
print("Loaded embeddings shape:", embeddings.shape)

Using checkpoint: logs/embeddings/embedding.ckpt-1
Variables in checkpoint:
  _CHECKPOINTABLE_OBJECT_GRAPH []
  embedding/.ATTRIBUTES/VARIABLE_VALUE [30, 384]
  save_counter/.ATTRIBUTES/VARIABLE_VALUE []
Loading variable 'embedding/.ATTRIBUTES/VARIABLE_VALUE' with shape [30, 384] from checkpoint...
Loaded embeddings shape: (30, 384)


In [11]:
import faiss
import numpy as np
import pandas as pd

# Cargar metadata
metadata = pd.read_csv("logs/embeddings/metadata.tsv", sep="\t")

# Normalizar embeddings
embeddings_norm = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# Crear índice FAISS
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings_norm.astype("float32"))

def top_k_products(query_idx, k=5):
    query_vec = embeddings_norm[query_idx].astype("float32").reshape(1, -1)
    D, I = index.search(query_vec, k=50)  # pedimos más resultados y luego filtramos

    productos = []
    for dist, idx in zip(D[0], I[0]):
        if metadata.iloc[idx]["tipo"] == "producto":
            productos.append((dist, idx))
        if len(productos) >= k:
            break

    return productos

# Ejemplo: recomendación para el cliente 0
cliente_idx = 0
print("Cliente:", metadata.iloc[cliente_idx]["texto_completo"])
print("\nTop-5 productos similares:")
for dist, idx in top_k_products(cliente_idx, k=5):
    row = metadata.iloc[idx]
    print(f" - {row['label']} | score={dist:.4f}")


Cliente: Ana Martínez Madrid España Smartphone Nexus 5G Laptop Gamer Pro Auriculares Inalámbricos X Camiseta Deportiva Ultralight Zapatillas Urbanas Fit Smartphone Nexus 5G Laptop Gamer Pro Laptop Gamer Pro Laptop Gamer Pro Laptop Gamer Pro Smartphone Nexus 5G El Smartphone Nexus 5G redefine la experiencia móvil. Su pantalla OLED de 6.7 pulgadas con una frecuencia de 120Hz proporciona colores vibrantes y negros profundos. La cámara principal de 108MP, junto con un lente ultra gran angular y un teleobjetivo, captura fotos de calidad profesional en cualquier condición de iluminación. Equipado con un procesador Qualcomm Snapdragon 8 Gen 2, 12GB de RAM y una batería de 5000 mAh, este teléfono ofrece un rendimiento fluido y una autonomía excepcional. Soporta carga rápida de 65W. Laptop de alto rendimiento para gaming y creación de contenido. Cuenta con un procesador Intel Core i9 de última generación, 32GB de RAM DDR5 y una tarjeta gráfica NVIDIA RTX 4080. Su pantalla de 16 pulgadas con tas

In [12]:
from sentence_transformers import SentenceTransformer

print("SentenceTransformers disponible. Usando modelo multilingüe.")
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
st_model = SentenceTransformer(model_name)

/Users/erichuiza/Documents/pucp/miería web/dev/sesion-de-clase-06/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SentenceTransformers disponible. Usando modelo multilingüe.


In [14]:
def search_query(query_text, k=5, tipo="producto"):
    # generar embedding del query
    q_emb = st_model.encode([query_text], convert_to_numpy=True)
    q_emb = q_emb / np.linalg.norm(q_emb, axis=1, keepdims=True)

    # buscar en FAISS
    D, I = index.search(q_emb.astype("float32"), k=50)

    resultados = []
    for dist, idx in zip(D[0], I[0]):
        if metadata.iloc[idx]["tipo"] == tipo:
            row = metadata.iloc[idx]
            resultados.append((dist, row["label"], row["texto_completo"]))
        if len(resultados) >= k:
            break

    return resultados

In [39]:
# Ejemplo de uso:
query = "fotografía"
resultados = search_query(query, k=5, tipo="producto")

print(f"🔎 Query: {query}\n")
for score, label, texto in resultados:
    print(f"- {label} | score={score:.4f}\n  {texto[:100]}...")

🔎 Query: fotografía

- Cámara de Acción 4K | score=0.4897
  Cámara de Acción 4K...
- Camiseta Deportiva Ultralight | score=0.3590
  Camiseta Deportiva Ultralight...
- Auriculares Inalámbricos X | score=0.3174
  Auriculares Inalámbricos X...
- Mochila Urbana Tech | score=0.2968
  Mochila Urbana Tech...
- Zapatillas Urbanas Fit | score=0.2845
  Zapatillas Urbanas Fit...
